<a href="https://colab.research.google.com/github/berrygayo/Deep-learning-to-learn-from-the-founder-of-Keras/blob/master/3_4_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98_%EC%9D%B4%EC%A7%84_%EB%B6%84%EB%A5%98_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import imdb 

# 3.4.1 IMDB 데이터 셋
인터넷 영화 데이터베이스로부터 가져온 양극단의 리뷰 5만 개로 이루어진 데이터 
- 훈련 데이터 : 25000개 
- 테스트 데이터 : 25000개 
각각 50%는 부정, 50%는 긍정 

이 데이터는 전처리에 있어 각 리뷰(단어 시퀀스)가 숫자 시퀀스로 변환되어 있다. 
여기서 각 숫자는 사전에 있는 고유한 단어를 나타낸다. 
데이터셋의 전체 문서에 나타난 모든 단어에 고유한 번호를 부여한 목록을 어휘 사전 또는 사전이라고 부릅니다.

In [ ]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000) 
# num_words=10000 : 훈련 데이터에서 가장 자주 나타나는 단어 1만 개 사용 

In [ ]:
# 리뷰 - 각 리뷰는 단어 인덱스의 리스트( 단어 시퀀스가 인코딩 된 것)
#train_data[0]

In [ ]:
# 0 : 부정, 1 : 긍정 
train_labels[0]

In [ ]:
max([max(sequence) for sequence in train_data])
# 가장 자주 등장하는 단어 1만 개로 제한했기 때문에 단어 인덱스는 9999를 넘지 않는다. 

In [ ]:
# 숫자 시퀀스가 원래 영어 단어 어떻게 되어있는지 보기 
word_index=imdb.get_word_index() # word_index는 단어와 정수 인덱스를 매핑한 딕셔너리 
reverse_word_index=dict(
    [(value, key) for (key, value) in word_index.items()]) #정수 인덱스와 단어를 매핑하도록 뒤집는다 
decoded_review=' '.join(
    [reverse_word_index.get(i-3, '?') for i in train_data[0]]) # 리뷰를 디코딩 합니다. 0,1,2는 '패딩','문서 시작','사전에 없음'을 위한 인덱스 이므로 3을 뺍니다. 

In [ ]:
decoded_review

In [ ]:
# get : 딕셔너리 구조에서 name key에 맞는 value 값을 돌려 줌 
a = {'name':'pey', 'phone':'0119993323', 'birth': '1118'}
a.get('name','?') # key가 딕셔너리에 없다면 ? 를 반환함 
a.get('hi','?')

# 3.4.2 데이터 준비 
리스트 > 텐서로 변환 

1. 같은 길이가 되도록 리스트에 패딩(paddind)을 추가하고 (samples, sequence_length) 크기의 정수 텐서로 변환합니다.
그다음 이 정수 텐서를 다룰 수 있는 층을 신경망의 첫 번째 층으로 사용합니다(Embedding 층을 말하며, 나중에 자세히 다룬다)

2. 리스트를 원-학 인코딩(one-hot encoding)하여 0과 1의 벡터로 변환합니다. 
예를 들어 시퀀스 [3,5]를 인덱스 3과 5의 위치는 1이고 그 외는 모두 0인 10000차원의 벡터로 각자 변환합니다. 
그다음 부동 소수 벡터 데이터를 다룰 수 있는 Dense 층을 신경망의 첫 번째 층으로 사용합니다.

여기서는 두 번째 방식을 사용하고 이해를 돕기 위해 직접 데이터를 
원-핫 벡터로 만들겠습니다.

In [ ]:
import numpy as np 

def vectorize_sequence(sequences, dimension=10000):
    results=np.zeros((len(sequences), dimension)) #크기가 len(sequence)* dimension 원소가 0인 행렬을 만듬
    for i, sequence in enumerate(sequences):
        results[i,sequence]=1. # result[i] 에서 특정 인덱스의 위치를 1로 만든다
    return results 

x_train=vectorize_sequence(train_data)
x_test=vectorize_sequence(test_data)                

In [ ]:
x_train[0]

In [ ]:
#레이블은 쉽게 벡터로 바꿀 수 있다.
y_train=np.asarray(train_labels).astype('float32')
y_test=np.asarray(test_labels).astype('float32')

In [ ]:
y_train

# 3.4.3 신경망 모델 만들기 
입력 데이터가 벡터고 레이블은 스칼라(1 또는 0)입니다. 아마 앞으로 볼 수 있는 문제 중에서 가장 간단할 것입니다.
이런 문제에 잘 작동하는 네트워크 종류는 relu 활성화 함수를 사용한 완전 연결 층(즉 Danse(16, activation='relu'))을 그냥 쌓은 것입니다.
Dense 층에 전달한 매개변수(16)은 은닉 유닛(hidden unit)의 개수입니다. 하나의 은닉 유닛은 층이 나타내는 표현 공간에서 하나의 차원이 됩니다.

>     output=relu(dot(W, input) + b)

16개의 은닉 유닛이 있다는 것은 가중치 행렬 w의 크기가 (input_dimension, 16)이라는 뜻입니다. 
입력 데이터와 W를 점곱하면 입력 데이터가 16차원으로 표현된 공간으로 투영됩니다. 
(그리고 편향 벡터 b를 더하고 relu 연산을 적용합니다).
표현 공간의 차원을 '신경망이 내재된 표현을 학습할 때 가질 수 있는 자유도'로 이해할 수 있습니다.
은닉 유닛을 늘리면(표현 공간을 더 고차원으로 만들면) 신경망이 더욱 복잡한 표현을 학습할 수 있지만
계산 비용이 커지고 원하지 않는 패턴을 학습할 수도 있습니다. 
(훈련 데이터에서는 성능이 향상되지만 테스트 데이터에서는 그렇지 않은 패턴)

> Dense 층을 쌓을 때 두 가지 중요한 구조상의 결정이 필요 
- 얼마나 많은 층을 사용할 것인가?
- 각 층에 얼마나 많은 은닉 유닛을 둘 것인가?

4장에서 이런 결정을 하는 데 도움이 되는 일반적인 원리를 배운다. 당분간은 필자를 따라가자.
- 16개의 은닉 유닛을 가진 2개의 은닉층(relu - 0과 1 사이의 점수) 
- 현재 리뷰의 감정을 스칼라 값의 예측으로 출력하는 세번째 층 (sigmoid-[0,1]사이의 확률)

In [ ]:
from keras import models 
from keras import layers 

model=models.Sequential()
model.add(layers.Dense(16,activation='relu',input_shape=(10000,)))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

## 활성화 함수가 무엇인가요 ?  왜 필요한가요 ? 
relu와 같은 활성화 함수(또는 비선형(non-linearly))가 없다면 Dense 층은 선형적인 연산인 점곱과 덧셈 2개로 구성됩니다. 

> output=dot(W, input)+b 

그러므로 이 층은 입력에 대한 선형 변환(아핀 변환)만을 학습할 수 있습니다. 
이 층의 가설 공간은 입력 데이터를 16차원의 공간으로 바꾸는 가능한 모든 선형 변환의 집합입니다. 
이런 가설 공간은 매우 제약이 만 ㅎ으며, 선형 층을 깊게 쌓아도 여전히 하나의 선형 연산이기 때문에 층을 여러 개로 구성하는 장점이 없습니다.
즉 층을 추가해도 가설 공간이 확장되지 않습니다.
가설 공간을 풍부하게 만들어 층을 깊게 만드는 장점을 살리기 위해서는 비선형성 또는 활성화 함수를 추가해야 합니다.
relu는 딥러닝에서 가장 인기 있는 활성화 함수입니다.
이름은 조금 이상하지만 prelu, elu 등 비슷한 다른 함수들도 많습니다. 


마지막으로 손실 함수와 옵티마이저를 선택해야 합니다. 
이진 분류 문제고 신경망의 출력이 확률이기 때문에(네트워크의 끝에 시그모이드 활성화 함수를 사용한 하나의 유닛으로 된 층을 놓았습니다).
binary_crossentrophy 손실이 적합합니다.
이 함수가 유일한 선택은 아니고 mean_crossentrophy 도 사용할 수 있습니다.
확률을 출력하는 모델을 사용할 때는 크로스엔트로피가 최선의; 선택입니다.
크로스엔트로피는 정보 이론 분야에서 온 개념으로 확률 분포 간의 차이를 측정합니다. 여기에서는 원본 분포와 예측 분포 사이를 측정합니다.

다음은 rmsprop 옵티마이저와 binary_crkssentrophy 손실 함수로 모델을 설정하는 단계입니다.
훈련하는 동안 정확도를 사용하여 모니터링 하겠습니다. 

In [ ]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 옵티마이저의 매개변수를 바꾸거나 자신만의 손실 함수, 측정 함수를 전달하려면 아래처럼 객체를 직접 만듬 
from keras import optimizers 

model.compile(optimizer=optimizers.RMSprop(lr=0.001), 
              loss='binary_crossentrophy',
              metrics=['accuracy'])

In [ ]:
from keras import losses 
from keras import metrics 

model.compile(optimizer=optimizers.RMSprop(lr=0.001), 
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

# 3.4.4 훈련 검증 
훈련하는 동안 처음 본 데이터에 대한 모델의 정확도를 측정하기 위해서는 원본 훈련 데이터에서 10000의 샘플을 떼어 검증 세트를 만들어야 한다.


In [ ]:
x_val=x_train[:10000]
partial_x_train=x_train[10000:]
y_val=y_train[:10000]
partial_y_train=y_train[10000:]

이제 모델을 512개의 샘플씩 미니 배치를 만들어 20번의 에포크 동안 훈련시킵니다.
x_train과 y_train 텐서에 있는 모든 샘플에 대해 20번 반복 )
동시에 따로 떼어놓은 1만 개의 샘플에서 손실과 정확도를 측정할 것입니다.
이렇게 하면 validation_data 매개변수에 검증 데이터를 전달해야합니다. 

In [ ]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['acc'])

history=model.fit(partial_x_train,
                  partial_y_train,
                  epochs=20,
                  batch_size=512,
                  validation_data=(x_val, y_val))

model.fit() 매서드는 History 객체를 반환합니다.
이 객체는 훈련하는 동안 발생한 모든 정보를 담고 있는 딕셔너리인 history 속성을 가지고 있습니다 

In [ ]:
history_dict=history.history
history_dict.keys()

이 딕셔너리는 훈련과 검증하는 동안 모니터링할 측정 지표당 하나씩 모두 4개의 항목을 담고 있습니다.
이어지는 두 목록에서 맥플롯립을 사용하여 훈련과 검증 데이터에 대한 손실과 정확도를 그리겠습니다. 

In [ ]:
import matplotlib.pyplot as plt 

history_dict=history.history
loss=history_dict['loss']
val_loss=history_dict['val_loss']

epochs=range(1,len(loss)+1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf() #그래프를 초기화 

acc=history_dict['acc']
val_acc=history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

## 해석 
훈련데이터는 에포크마다 감소, 증가 하지만 
검증 데이터에서는 이와 같지 않다.
네 번째 에포크에서 그래프가 역전되는걸 볼 수 있다.
> 과대적합 

두 번째 에포크부터 훈련 데이터에 과도하게 최적화되어 훈련 데이터에 특화된 표현을 학습하므로 훈련 세트 이외의 데이터에는 일반화되지 못합니다.

이런 경우에 과대적합을 방지하기 위해서 세 번재 에포크 이후에 훈련을 중지할 수 있습니다.
일반적으로 4장에서 보게 될 과대 적합을 완화하는 다양한 종류의 기술을 사용할 수 있습니다.
처음부터 다시 새로운 신경망을 네 번의 에포크 동안만 훈련하고 테스트 데이터에서 평가해보겠습니다. 


In [ ]:
model=models.Sequential()
model.add(layers.Dense(16, activation='relu',input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=4, batch_size=512)
results=model.evaluate(x_test,y_test)

In [ ]:
results #loss, 정확도 

# 3.4.5 훈련된 모델로 새로운 데이터에 대해 예측하기 

In [ ]:
model.predict(x_test)